# Final Project

### COSC 410B: Spring 2025, Colgate University



In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

data=pd.read_csv("geneExpressionDataKidney.csv")
target=data["label"]
features=data.drop(columns=["label","sample_id"])
trainSet, testSet, trainClasses, testClasses=train_test_split(features, target, test_size=0.2, stratify=target, random_state=111)

def trainEvaluateModel(model, parameters, varianceThreshold=0.01, topGenesToKeep=1000):
    variance=VarianceThreshold(varianceThreshold)#There are several ways to conduct feature selection through sklearn, which have been documented here: https://scikit-learn.org/stable/modules/feature_selection.html
    topGenes=SelectKBest(f_classif, k=topGenesToKeep)
    pipe=Pipeline([("log1p", FunctionTransformer(np.log1p, validate=False)),("varianceFilter",variance),("scaler", StandardScaler()),("topKFeatures",topGenes),("model",model)])#sklearn pipelines makes feature selection convenient. We learned about them here: https://www.geeksforgeeks.org/what-is-exactly-sklearnpipelinepipeline/
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=111)#We have to call StratifiedKFold() here instead of just doing cv=5 in GridSearchCV() to ensure reproducibility, shuffling, and stratification 
    gridSearch=GridSearchCV(estimator=pipe, param_grid=parameters, scoring='neg_brier_score', cv=5, n_jobs=-1)
    gridSearch.fit(trainSet, trainClasses)#Hyperparameter tuning
    bestModel=gridSearch.best_estimator_

    #Evaluating Predicted Probabilities
    probabilities=bestModel.predict_proba(testSet)[:, 1]#Gathering the probabilities assigned to each sample
    print("Performance on the Test Set:")
    print(f"Brier Score: {brier_score_loss(testClasses, probabilities)}")
    print(f"Cross-entropy Loss: {log_loss(testClasses, probabilities)}")

    #Evaluating Predicted Labels
    testPredictions=(probabilities >= 0.5).astype(int)#Converting the probabilities into labels with a threshold of 0.5
    print(f"Accuracy: {accuracy_score(testClasses, testPredictions)}")
    print(f"Precision: {precision_score(testClasses, testPredictions)}")
    print(f"Recall: {recall_score(testClasses, testPredictions)}")
    print(f"F1 Score: {f1_score(testClasses, testPredictions)}\n")

    #Interpreting the Model (COMMENT THIS OUT IF NOT EVALUATING A LINEAR MODEL)
    print("The Top Genes that Influence the Model's Predictions:")
    weights=bestModel.named_steps["model"].coef_[0]#Extract the model's weights for the top features (the only weights it has since it was only trained on the top features). The "bestModel.named_steps["X"]" notation is just accessing the X step of the pipeline
    unfilteredVarianceFeatures=features.columns[bestModel.named_steps["varianceFilter"].get_support()]#Extract the top features the model was trained on
    topFeatures=unfilteredVarianceFeatures[bestModel.named_steps["topKFeatures"].get_support()]
    topWeights=pd.DataFrame({"Gene": topFeatures, "Weight": weights})
    topWeights["Absolute Weight"]=topWeights["Weight"].abs()
    topWeights=topWeights.sort_values("Absolute Weight", ascending=False)#Sort the data frame by the absolute weight column so that the most influential features come first
    print(topWeights.head(10).to_string(index=False))#Print out the top ten most influential features



In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from collections import Counter
from collections import defaultdict
from sklearn.utils import resample

data=pd.read_csv("geneExpressionDataBreast.csv")
target=data["label"]
features=data.drop(columns=["label","sample_id"])
trainSet, testSet, trainClasses, testClasses=train_test_split(features, target, test_size=0.2, stratify=target, random_state=111)

def retrieveTopGenes(model, parameters, trainingSet, trainingClasses, varianceThreshold=0.01, topGenesToKeep=1000):
    variance=VarianceThreshold(varianceThreshold)#There are several ways to conduct feature selection through sklearn, which have been documented here: https://scikit-learn.org/stable/modules/feature_selection.html
    topGenes=SelectKBest(f_classif, k=topGenesToKeep)
    pipe=Pipeline([("log1p", FunctionTransformer(np.log1p, validate=False)),("varianceFilter",variance),("scaler", StandardScaler()),("topKFeatures",topGenes),("model",model)])#sklearn pipelines makes feature selection convenient. We learned about them here: https://www.geeksforgeeks.org/what-is-exactly-sklearnpipelinepipeline/
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=111)#We have to call StratifiedKFold() here instead of just doing cv=5 in GridSearchCV() to ensure reproducibility, shuffling, and stratification 
    gridSearch=GridSearchCV(estimator=pipe, param_grid=parameters, scoring='neg_brier_score', cv=cv, n_jobs=-1)
    gridSearch.fit(trainingSet, trainingClasses)#Hyperparameter tuning
    bestModel=gridSearch.best_estimator_
    
    probabilities=bestModel.predict_proba(testSet)[:, 1]#Gathering the probabilities assigned to each sample
    predictions=(probabilities >= 0.5).astype(int)#Converting the probabilities into labels with a threshold of 0.5  
    metrics={"accuracy": accuracy_score(testClasses, predictions), "precision": precision_score(testClasses, predictions), "recall": recall_score(testClasses, predictions), "f1": f1_score(testClasses, predictions)}
 
    weights=bestModel.named_steps["model"].coef_[0]#Extract the model's weights for the top features (the only weights it has since it was only trained on the top features). The "bestModel.named_steps["X"]" notation is just accessing the X step of the pipeline
    unfilteredVarianceFeatures=trainingSet.columns[bestModel.named_steps["varianceFilter"].get_support()]#Extract the top features the model was trained on
    topFeatures=unfilteredVarianceFeatures[bestModel.named_steps["topKFeatures"].get_support()]
    topWeights=pd.DataFrame({"Gene": topFeatures, "Weight": weights})
    topWeights["Absolute Weight"]=topWeights["Weight"].abs()
    topWeights=topWeights.sort_values("Absolute Weight", ascending=False)#Sort the data frame by the absolute weight column so that the most influential features come first
    return metrics, topWeights.head(50).reset_index(drop=True)
 
 
 
  

In [4]:
#Logistic Regression
model=LogisticRegression(max_iter=1000)
parameters={
    "model__C":[0.1, 1],
    "model__penalty":["l2"],
    "model__solver":["lbfgs"],
}
#trainEvaluateModel(model, parameters)

runs=100
topGeneTally=Counter()
signTally=defaultdict(Counter)
totMetrics={"accuracy": 0, "precision": 0, "recall": 0, "f1": 0}
for i in range(runs):
    trainSetResample, trainClassesResample=resample(trainSet, trainClasses, replace=True, n_samples=int(0.8*len(trainClasses)), random_state=111+i)
    metrics, topGenes=retrieveTopGenes(model, parameters, trainSetResample,trainClassesResample)
    for _, row in topGenes.iterrows():
        gene=row["Gene"]
        weight=row["Weight"]
        topGeneTally[gene]+=1
        sign="+" if weight>0 else "-"
        signTally[gene][sign]+= 1
    
    for metric in metrics:
        totMetrics[metric]+=metrics[metric]
        
geneFrequency={gene: topGeneTally[gene]/runs for gene in features.columns}
#print(topGeneTally)
#print(geneFrequency)
consistentlySelectedGenes=sorted([gene for gene,frequency in geneFrequency.items() if frequency>=0.80], key=lambda gene: geneFrequency[gene], reverse=True)


print("Average Classification Metrics Over the 100 runs:")
for metric in totMetrics:
    print(f"  {metric}: {totMetrics[metric]/runs}")

print(f"\n{len(consistentlySelectedGenes)} gene(s) selected ≥ 80% of runs:")
for gene in consistentlySelectedGenes:
    rate=geneFrequency[gene]*100
    print(f"  {gene}: {rate}%. Signs across the runs: (+){signTally[gene]["+"]} and (-){signTally[gene]["-"]}")


Average Classification Metrics Over the 100 runs:
  accuracy: 1.0
  precision: 1.0
  recall: 1.0
  f1: 1.0

15 gene(s) selected ≥ 80% of runs:
  ENSG00000149380.12: 100.0%. Signs across the runs: (+)100 and (-)0
  ENSG00000172061.9: 99.0%. Signs across the runs: (+)99 and (-)0
  ENSG00000155886.11: 98.0%. Signs across the runs: (+)98 and (-)0
  ENSG00000160161.9: 97.0%. Signs across the runs: (+)97 and (-)0
  ENSG00000164932.13: 97.0%. Signs across the runs: (+)97 and (-)0
  ENSG00000101463.6: 94.0%. Signs across the runs: (+)94 and (-)0
  ENSG00000182492.16: 94.0%. Signs across the runs: (+)94 and (-)0
  ENSG00000261327.5: 93.0%. Signs across the runs: (+)93 and (-)0
  ENSG00000122641.11: 91.0%. Signs across the runs: (+)91 and (-)0
  ENSG00000261039.3: 87.0%. Signs across the runs: (+)87 and (-)0
  ENSG00000123500.10: 85.0%. Signs across the runs: (+)85 and (-)0
  ENSG00000180044.5: 84.0%. Signs across the runs: (+)84 and (-)0
  ENSG00000104415.14: 81.0%. Signs across the runs: (+)81